In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/NormalNet')


import sys
sys.path.append('/content/drive/My Drive/NormalNet')

In [ ]:
import numpy as np
from keras.callbacks import EarlyStopping 
import pandas as pd 
import matplotlib.pyplot as plt 
import shutil 
from os import listdir, makedirs, getcwd, remove 
from os.path import isfile, join, abspath, exists, isdir, expanduser 
import matplotlib.image as mimg
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array 
import tensorflow as tf 
from keras import layers 
from keras import models 
from keras import optimizers 
from sklearn.preprocessing import LabelEncoder 
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array 

In [ ]:
def plot_and_save_image(running_corrects,val_running_corrects, running_loss,val_running_loss, title, direc):
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,5))
    fig.suptitle(title)
    ax1.plot(running_corrects)
    ax1.plot(val_running_corrects)
    ax1.set_title('model accuracy')

    ax1.legend(['train', 'test'], loc='upper left')
    ax2.plot(running_loss)
    ax2.plot(val_running_loss)
    ax2.set_title('model loss')

    ax2.legend(['train', 'test'], loc='upper left')
    
    c = ['accuracy','loss']
    j = 0
    for ax in [ax1,ax2]:
        
        ax.set(xlabel='epochs', ylabel=c[j])
        j = j+1
    ax1.grid()
    ax2.grid()
    plt.savefig(direc)

In [ ]:
batch_size = 32
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '../dataset/data/train',
        target_size=(100, 100),  
        batch_size=batch_size,
        class_mode='categorical')  

validation_generator = test_datagen.flow_from_directory(
        '../dataset/data/valid',
        target_size=(100, 100),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = False
)

# FIRST SIMPLE CNN 

In [ ]:

from keras.models import Sequential
from keras.layers import Dense

from keras import optimizers
from keras import initializers
from keras.layers import Conv2D,Flatten,Dropout, MaxPooling2D,AveragePooling2D,BatchNormalization
norm = initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)
model = Sequential()
#first convolutional layer
model.add(Conv2D(filters=8,kernel_size=(3,3),padding = 'valid',activation='relu',input_shape=(100,100,3)))
#model.add(BatchNormalization(axis = 1)
model.add(MaxPooling2D(pool_size=2,strides=2))
    
#model.add(Dropout(0.3))

#secondo convolutional layer
model.add(Conv2D(filters=16,kernel_size=(3,3),strides=2,padding='valid',activation='relu'))
#model.add(BatchNormalization(axis = 1))
model.add(MaxPooling2D(pool_size=2,strides=2))
    
model.add(Dropout(0.3))

#third convolutional layer
model.add(Conv2D(filters=32,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(Flatten())
model.add(Dense(128,activation='relu',kernel_initializer=norm,bias_initializer='zeros'))
    
model.add(BatchNormalization(axis = 1))
model.add(Dense(20,activation='softmax'))




In [ ]:
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['acc'])
callback = [EarlyStopping(monitor='val_acc', patience=10)]

In [ ]:
history = model.fit_generator(
          train_generator,
          epochs=100,
          validation_data=validation_generator , callbacks = callback)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
plot_and_save_image(acc,val_acc,loss,val_loss,'accuracy and loss ','risultati/scratchnet/model_A/acc_loss.jpg')

In [ ]:
# create labels vector 
def from_dummy_to_labels(dummy_vec):
    res = np.zeros(dummy_vec.shape[0])
    for i in range(dummy_vec.shape[0]):
        tmp = list(dummy_vec[i])
        res[i] = tmp.index(1)
    return res


prediction = np.zeros(shape = (449))
labels_dummy = np.zeros(shape = (449,20))
batch_size = 32
generator = ImageDataGenerator(rescale = 1./255).flow_from_directory('../dataset/data/valid/', target_size = (100, 100), 
                                                                            batch_size = batch_size, class_mode = 'categorical',shuffle = False)

i = 0

for input_batch, labels_batch in generator:
        
      features_batch = model.predict(input_batch)
      y_pred = np.argmax(features_batch, axis=1)
      #print(y_pred)
      #features[i*batch_size : (i + 1)*batch_size] = features_batch 
      labels_dummy[i*batch_size : (i + 1)*batch_size] = labels_batch
      prediction[i*batch_size : (i + 1)*batch_size] = y_pred
      #features.append(features_batch)
      #labels_dummy.append(labels_batch)
        
      i = i + 1
        
      if i*batch_size >= 449:
          break
labels = from_dummy_to_labels(labels_dummy)

#predictions = model.predict(validation_generator)


#tf.math.confusion_matrix(
#    labels, predictions, num_classes=None, weights=None, dtype=tf.dtypes.int32,
#    name=None
#)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print('Confusion Matrix')
conf_mat = confusion_matrix(prediction, labels, normalize= 'true')
print('mean: ',np.mean(np.diag(conf_mat)))
print('std: ',np.std(np.diag(conf_mat)))
print('Classification Report')
target_names = ['Arthonia_radiata','Caloplaca_cerina','Candelariella_reflexa','Candelariella_xanthostigma','Chrysothrix_candelaris','Flavoparmelia_caperata','Gyalolechia_flavorubescens','Hyperphyscia_adglutinata'
        ,'Lecanora_argentata','Lecanora_chlarotera','Lecidella_elaeochroma','Melanelixia_glabratula'
        ,'Phaeophyscia_orbicularis','Physcia_biziana','Physconia_grisea','Ramalina_farinacea','Ramalina_fastigiata','Xanthomendoza_fallax','Xanthomendoza_fulva','flavoparmenia_soredians']

#print(classification_report(labels, prediction, target_names=target_names))

In [ ]:
Y_pred = model.predict(generator, 449 // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(labels, y_pred))
print('Classification Report')
target_names = ['Arthonia_radiata','Caloplaca_cerina','Candelariella_reflexa','Candelariella_xanthostigma','Chrysothrix_candelaris','Flavoparmelia_caperata','Gyalolechia_flavorubescens','Hyperphyscia_adglutinata'
        ,'Lecanora_argentata','Lecanora_chlarotera','Lecidella_elaeochroma','Melanelixia_glabratula'
        ,'Phaeophyscia_orbicularis','Physcia_biziana','Physconia_grisea','Ramalina_farinacea','Ramalina_fastigiata','Xanthomendoza_fallax','Xanthomendoza_fulva','flavoparmenia_soredians']

print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

# SECOND MODEL


In [ ]:
from keras.layers import Conv2D, MaxPooling2D

model3 = models.Sequential()

model3.add(Conv2D(128, kernel_size=3, activation='relu',padding='same', input_shape=(100,100,3)))
#model3.add(layers.Dropout(0.5))
model3.add(MaxPooling2D(padding = 'same'))


model3.add(Conv2D(256, kernel_size=3, activation='relu',padding='same'))
#model3.add(layers.Dropout(0.5))
model3.add(MaxPooling2D(padding = 'same'))

model3.add(Conv2D(256, kernel_size=3, activation='relu',padding='same'))
#model3.add(layers.Dropout(0.5))
model3.add(MaxPooling2D(padding = 'same'))


model3.add(Conv2D(512, kernel_size=3, activation='relu',padding='same'))
#model3.add(layers.Dropout(0.5))
model3.add(MaxPooling2D(padding = 'same'))

model3.add(layers.BatchNormalization())
model3.add(layers.Flatten())
model3.add(layers.Dense(128, activation='relu'))
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(20, activation='softmax'))


In [ ]:

model3.summary()

In [ ]:
from keras.callbacks import EarlyStopping
model3.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['acc'])
callback = [EarlyStopping(monitor='val_acc', patience=10)]


history = model3.fit_generator(
          train_generator,
          epochs=100,
          validation_data=validation_generator , callbacks = callback)

In [ ]:

prediction = np.zeros(shape = (449))
labels_dummy = np.zeros(shape = (449,20))
batch_size = 32
generator = ImageDataGenerator(rescale = 1./255).flow_from_directory('../dataset/data/valid/', target_size = (100, 100), 
                                                                            batch_size = batch_size, class_mode = 'categorical',shuffle = False)

i = 0

for input_batch, labels_batch in generator:
        
      features_batch = model3.predict(input_batch)
      y_pred = np.argmax(features_batch, axis=1)
      #print(y_pred)
      #features[i*batch_size : (i + 1)*batch_size] = features_batch 
      labels_dummy[i*batch_size : (i + 1)*batch_size] = labels_batch
      prediction[i*batch_size : (i + 1)*batch_size] = y_pred
      #features.append(features_batch)
      #labels_dummy.append(labels_batch)
        
      i = i + 1
        
      if i*batch_size >= 449:
          break
labels = from_dummy_to_labels(labels_dummy)


from sklearn.metrics import classification_report, confusion_matrix
#Y_pred = model.predict(validation_features, 449 // batch_size+1)
#y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
conf_mat = confusion_matrix(prediction, labels, normalize= 'true')
print('mean: ',np.mean(np.diag(conf_mat)))
print('std: ',np.std(np.diag(conf_mat)))
print('Classification Report')
target_names = ['Arthonia_radiata','Caloplaca_cerina','Candelariella_reflexa','Candelariella_xanthostigma','Chrysothrix_candelaris','Flavoparmelia_caperata','Gyalolechia_flavorubescens','Hyperphyscia_adglutinata'
        ,'Lecanora_argentata','Lecanora_chlarotera','Lecidella_elaeochroma','Melanelixia_glabratula'
        ,'Phaeophyscia_orbicularis','Physcia_biziana','Physconia_grisea','Ramalina_farinacea','Ramalina_fastigiata','Xanthomendoza_fallax','Xanthomendoza_fulva','flavoparmenia_soredians']




In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
plot_and_save_image(acc,val_acc,loss,val_loss,'accuracy and loss ','risultati/scratchnet/model_C/acc_loss.jpg')

# SAME of VGG FOR FEATURES EXTRACTION + DENSE LAYERS 

NO PRE-TRAINING

In [ ]:
from keras.applications import VGG16
conv_base = VGG16(weights = None, include_top = False, input_shape = (100, 100, 3))

In [ ]:
modelvgg = models.Sequential()
modelvgg.add(conv_base)
modelvgg.add(layers.Flatten())
modelvgg.add(layers.Dense(512, activation='relu'))
modelvgg.add(layers.Dropout(0.5))
modelvgg.add(layers.Dense(128, activation='relu'))
modelvgg.add(layers.Dense(20, activation='softmax'))

modelvgg.layers[0].trainable = True

In [ ]:
modelvgg.summary()

In [ ]:
from keras.callbacks import EarlyStopping
modelvgg.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['acc'])
callback = [EarlyStopping(monitor='val_acc', patience=10)]


history = modelvgg.fit_generator(
          train_generator,
          epochs=150,
          validation_data=validation_generator , callbacks = callback)

In [ ]:
# create labels vector 
def from_dummy_to_labels(dummy_vec):
    res = np.zeros(dummy_vec.shape[0])
    for i in range(dummy_vec.shape[0]):
        tmp = list(dummy_vec[i])
        res[i] = tmp.index(1)
    return res



prediction = np.zeros(shape = (449))
labels_dummy = np.zeros(shape = (449,20))
batch_size = 32
generator = ImageDataGenerator(rescale = 1./255).flow_from_directory('../dataset/data/valid/', target_size = (100, 100), 
                                                                            batch_size = batch_size, class_mode = 'categorical',shuffle = False)

i = 0

for input_batch, labels_batch in generator:
        
      features_batch = modelvgg.predict(input_batch)
      y_pred = np.argmax(features_batch, axis=1)
      #print(y_pred)
      #features[i*batch_size : (i + 1)*batch_size] = features_batch 
      labels_dummy[i*batch_size : (i + 1)*batch_size] = labels_batch
      prediction[i*batch_size : (i + 1)*batch_size] = y_pred
      #features.append(features_batch)
      #labels_dummy.append(labels_batch)
        
      i = i + 1
        
      if i*batch_size >= 449:
          break
labels = from_dummy_to_labels(labels_dummy)


from sklearn.metrics import classification_report, confusion_matrix
#Y_pred = model.predict(validation_features, 449 // batch_size+1)
#y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
conf_mat = confusion_matrix(prediction, labels, normalize= 'true')
print('mean: ',np.mean(np.diag(conf_mat)))
print('std: ',np.std(np.diag(conf_mat)))
print('Classification Report')
target_names = ['Arthonia_radiata','Caloplaca_cerina','Candelariella_reflexa','Candelariella_xanthostigma','Chrysothrix_candelaris','Flavoparmelia_caperata','Gyalolechia_flavorubescens','Hyperphyscia_adglutinata'
        ,'Lecanora_argentata','Lecanora_chlarotera','Lecidella_elaeochroma','Melanelixia_glabratula'
        ,'Phaeophyscia_orbicularis','Physcia_biziana','Physconia_grisea','Ramalina_farinacea','Ramalina_fastigiata','Xanthomendoza_fallax','Xanthomendoza_fulva','flavoparmenia_soredians']


print(classification_report(prediction, labels, target_names=target_names))


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
plot_and_save_image(acc,val_acc,loss,val_loss,'accuracy and loss ','risultati/scratchnet/model_vgg/acc_loss.jpg')